In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import os
from os.path import join
import time

%load_ext autoreload
%autoreload 2

In [2]:
OUT_PATH='/n/groups/price/martin/LDSPEC_data/UKBB_trait_rep287K'
DATA_PATH='/n/groups/price/martin/LDSPEC_data/UKBB_trait_from_Steven'

### Load data

In [3]:
# Get the ID list
df_fam = pd.read_csv('/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001/unrelated_337K_rep287K.txt',
                     sep=' ', index_col=False)
df_fam.columns = ['FID', 'IID']
id_list = list(df_fam['IID'])
print('n_sample=%d' % len(id_list))

n_sample=287047


In [4]:
# 64 independent traits
xl = pd.ExcelFile(DATA_PATH+'/Description_080419.xlsx')
df_64_inde_trait = xl.parse('64 Recommended indepent traits', )
inde_trait_list = list(df_64_inde_trait['Trait_Identifier'])

In [5]:
# UKBB phen file
df_phen=pd.read_csv(DATA_PATH+'/UKB_v3.061518.tab', sep='\t', index_col=False)
df_cov=pd.read_csv(DATA_PATH+'/ukb4777.processed_and_post.plinkPCs.tab.gz', 
                   sep='\t', index_col=False, compression='gzip')
df_phen_bc=pd.read_csv(DATA_PATH+'/UKB_biochemistry.051619.tab',
                       sep='\t', index_col=False)
df_cov_bc=pd.read_csv(DATA_PATH+'/UKB_biochemistry.051619.cov',
                       sep='\t', index_col=False)

### trait_list, .phen, and .cov files

In [6]:
trait_list = sorted([x for x in df_phen.columns if x not in ['FID', 'IID']])
print('# trait_list=%d'%len(trait_list))
temp_df = df_phen.loc[df_phen['FID'].isin(id_list)].copy()
# trait_list = [x for x in trait_list if (~temp_df[x].isna()).sum()>1e5] # filter for N>1e5
# print('# Remove N<1e5 traits, trait_list=%d'%len(trait_list))
trait_list_indpt = [x for x in trait_list if 'UKB_460K.%s'%x in inde_trait_list]
print('# Restrict to 64 indpt traits, trait_list_indpt=%d'%len(trait_list_indpt))

trait_list_bc = sorted([x for x in df_phen_bc.columns if x not in ['FID', 'IID']])
print('# trait_list_bc=%d'%len(trait_list_bc))
temp_df = df_phen_bc.loc[df_phen_bc['FID'].isin(id_list)].copy()
# trait_list_bc = [x for x in trait_list_bc if (~temp_df[x].isna()).sum()>1e5]
# print('# Remove N<1e5 traits, trait_list_bc=%d'%len(trait_list_bc))
trait_list_bc_indpt = [x for x in trait_list_bc if 'UKB_460K.%s'%x in inde_trait_list]
print('# Restrict to 64 indpt traits, trait_list_bc_indpt=%d'%len(trait_list_bc_indpt))

with open(OUT_PATH+'/trait_list.txt', 'w') as f:
    for x in trait_list:
        f.write('%s\n'%x)
with open(OUT_PATH+'/trait_list_indpt.txt', 'w') as f:
    for x in trait_list_indpt:
        f.write('%s\n'%x)
with open(OUT_PATH+'/trait_list_other.txt', 'w') as f:
    for x in [x for x in trait_list if x not in trait_list_indpt]:
        f.write('%s\n'%x)
        
with open(OUT_PATH+'/trait_list_bc.txt', 'w') as f:
    for x in trait_list_bc:
        f.write('%s\n'%x)
with open(OUT_PATH+'/trait_list_bc_indpt.txt', 'w') as f:
    for x in trait_list_bc_indpt:
        f.write('%s\n'%x)
with open(OUT_PATH+'/trait_list_bc_other.txt', 'w') as f:
    for x in [x for x in trait_list_bc if x not in trait_list_bc_indpt]:
        f.write('%s\n'%x)
        
with open(OUT_PATH+'/trait_list_all.txt', 'w') as f:
    for x in trait_list+trait_list_bc:
        f.write('%s\n'%x)
with open(OUT_PATH+'/trait_list_all_indpt.txt', 'w') as f:
    for x in trait_list_indpt+trait_list_bc_indpt:
        f.write('%s\n'%x)
with open(OUT_PATH+'/trait_list_all_other.txt', 'w') as f:
    for x in [x for x in trait_list+trait_list_bc if x not in trait_list_indpt+trait_list_bc_indpt]:
        f.write('%s\n'%x)

# trait_list=93
# Restrict to 64 indpt traits, trait_list_indpt=23
# trait_list_bc=30
# Restrict to 64 indpt traits, trait_list_bc_indpt=9


In [7]:
# Write phen file
for trait in trait_list:
    temp_df = df_phen.loc[df_phen['FID'].isin(id_list), ['FID', 'IID', trait]].copy()
    temp_df.dropna(inplace=True)
    temp_df.to_csv(OUT_PATH + '/%s.phen'%trait, sep='\t', header=True, index=False)
    print('%s, n_sample=%d' % (trait, temp_df.shape[0]))
    
for trait in trait_list_bc:
    temp_df = df_phen_bc.loc[df_phen_bc['FID'].isin(id_list), ['FID', 'IID', trait]].copy()
    temp_df.dropna(inplace=True)
    temp_df.to_csv(OUT_PATH + '/%s.phen'%trait, sep='\t', header=True, index=False)
    print('%s, n_sample=%d' % (trait, temp_df.shape[0]))

blood_EOSINOPHIL_COUNT, n_sample=275025
blood_HIGH_LIGHT_SCATTER_RETICULOCYTE_COUNT, n_sample=273550
blood_LYMPHOCYTE_COUNT, n_sample=277523
blood_MEAN_CORPUSCULAR_HEMOGLOBIN, n_sample=277416
blood_MEAN_PLATELET_VOL, n_sample=278307
blood_MEAN_SPHERED_CELL_VOL, n_sample=273575
blood_MONOCYTE_COUNT, n_sample=276702
blood_PLATELET_COUNT, n_sample=277792
blood_PLATELET_DISTRIB_WIDTH, n_sample=277991
blood_RBC_DISTRIB_WIDTH, n_sample=276736
blood_RED_COUNT, n_sample=278308
blood_WHITE_COUNT, n_sample=277878
bmd_HEEL_TSCOREz, n_sample=278709
body_BALDING1, n_sample=131938
body_BALDING4, n_sample=131938
body_BMIz, n_sample=286117
body_CORNEAL_HYSTERESIS_LEFT, n_sample=59675
body_CORNEAL_HYSTERESIS_RIGHT, n_sample=59760
body_HEIGHTz, n_sample=286431
body_LEFT_HANDED, n_sample=286984
body_WHRadjBMIz, n_sample=286507
bp_DIASTOLICadjMEDz, n_sample=264343
bp_SYSTOLICadjMEDz, n_sample=264343
cancer_ALL, n_sample=287044
cancer_BREAST, n_sample=287044
cancer_MELANOMA, n_sample=287044
cancer_PROSTATE

### .resid.phen file

In [8]:
cov_list = ['cov_ASSESS_CENTER', 'cov_GENO_ARRAY', 'cov_SEX']
qcov_list = ['cov_AGE', 'cov_AGE_SQ'] + ['PC%d'%x for x in np.arange(1,21)] 

# Categorical covaraites 
temp_df_cov = df_cov[['FID', 'IID'] + cov_list]
temp_df_cov.index = temp_df_cov['IID']
temp_df_cov = pd.get_dummies(temp_df_cov[cov_list], columns=cov_list, drop_first=True)

# df_reg
df_reg = df_cov[['FID', 'IID'] + qcov_list]
df_reg.index = df_reg['IID']
df_reg = df_reg.join(temp_df_cov)
PC_list = [x for x in df_reg.columns if 'PC' in x]
df_reg[PC_list] *= np.sqrt(df_reg.shape[0])
df_reg['const'] = 1
df_reg.drop(['FID', 'IID'], axis=1, inplace=True)
df_reg = df_reg.astype(dtype=np.float32)

# Compute residual
for trait in trait_list:
    df_phen = pd.read_csv(OUT_PATH+'/%s.phen'%trait, sep='\t', header=0, index_col=False)
    df_phen.index = df_phen['IID']    
    sample_list = sorted(set(df_phen.index) & set(df_reg.index))
    print('%s, n_sample=%d, n_sample_retain=%d' % (trait, df_phen.shape[0], len(sample_list)))

    temp_df_reg = df_reg.loc[sample_list].copy()
    drop_list = [x for x in temp_df_reg if (temp_df_reg[x].std()<1e-2) & (x!='const')]
    if len(drop_list)>0:
        temp_df_reg = temp_df_reg.drop(columns=drop_list)
        print('    col dropped: %s'%', '.join(drop_list))
        
    v_y = df_phen.loc[sample_list][trait].values.astype(np.float32)
    mat_X = temp_df_reg.values.astype(np.float32)
    n_sample = v_y.shape[0]
    mat_xtx = np.dot(mat_X.T, mat_X)/n_sample
    mat_xty = np.dot(mat_X.T, v_y)/n_sample
    coef_ = np.linalg.solve(mat_xtx, mat_xty).reshape([-1])
    v_y_res = v_y - mat_X.dot(coef_)
    print('    var(v_y)=%0.4g, var(v_y_res)=%0.4g, R_square=%0.4g\n'
          %(v_y.var(), v_y_res.var(), 1- v_y_res.var()/v_y.var()))
    
    temp_df = pd.DataFrame( data = {
        "FID" : sample_list,
        "IID" : sample_list,
        trait : v_y_res / v_y_res.std()
    }
    )
    temp_df.to_csv(OUT_PATH + '/%s.resid.phen'%trait, sep='\t', header=True, index=False)

blood_EOSINOPHIL_COUNT, n_sample=275025, n_sample_retain=275025
    var(v_y)=0.9919, var(v_y_res)=0.9845, R_square=0.007464

blood_HIGH_LIGHT_SCATTER_RETICULOCYTE_COUNT, n_sample=273550, n_sample_retain=273550
    var(v_y)=0.9982, var(v_y_res)=0.9941, R_square=0.004061

blood_LYMPHOCYTE_COUNT, n_sample=277523, n_sample_retain=277523
    var(v_y)=0.987, var(v_y_res)=0.9787, R_square=0.008367

blood_MEAN_CORPUSCULAR_HEMOGLOBIN, n_sample=277416, n_sample_retain=277416
    var(v_y)=0.9526, var(v_y_res)=0.943, R_square=0.01002

blood_MEAN_PLATELET_VOL, n_sample=278307, n_sample_retain=278307
    var(v_y)=0.9922, var(v_y_res)=0.9889, R_square=0.00328

blood_MEAN_SPHERED_CELL_VOL, n_sample=273575, n_sample_retain=273575
    var(v_y)=0.9738, var(v_y_res)=0.9708, R_square=0.003101

blood_MONOCYTE_COUNT, n_sample=276702, n_sample_retain=276702
    var(v_y)=0.9899, var(v_y_res)=0.9858, R_square=0.004104

blood_PLATELET_COUNT, n_sample=277792, n_sample_retain=277792
    var(v_y)=0.9886, var(v_y_re

other_HEALTH_RATING, n_sample=286021, n_sample_retain=286021
    var(v_y)=0.5241, var(v_y_res)=0.5154, R_square=0.01659

other_MORNINGPERSON, n_sample=256333, n_sample_retain=256333
    var(v_y)=0.2335, var(v_y_res)=0.2312, R_square=0.009643

other_NAP_TIME, n_sample=286935, n_sample_retain=286935
    var(v_y)=0.2458, var(v_y_res)=0.2364, R_square=0.03831

other_NO_WHEAT, n_sample=286363, n_sample_retain=286363
    var(v_y)=0.02559, var(v_y_res)=0.02551, R_square=0.00309

other_SODIUM_URINE, n_sample=278183, n_sample_retain=278183
    var(v_y)=1903, var(v_y_res)=1741, R_square=0.08546

other_TIME_TV, n_sample=284991, n_sample_retain=284991
    var(v_y)=2.816, var(v_y_res)=2.64, R_square=0.06229

other_WALKING, n_sample=285435, n_sample_retain=285435
    var(v_y)=0.2401, var(v_y_res)=0.2337, R_square=0.02663

parents_ALZHEIMER, n_sample=279307, n_sample_retain=279307
    var(v_y)=0.1002, var(v_y_res)=0.09818, R_square=0.01998

parents_BOWEL_CANCER, n_sample=279307, n_sample_retain=27930

In [9]:
# trait_bc
cov_list = ['cov_ASSESS_CENTER', 'cov_GENO_ARRAY', 'cov_SEX']
qcov_list = ['cov_AGE', 'cov_AGE_SQ', 'biochemistry_dilutionfactor'] + ['PC%d'%x for x in np.arange(1,21)] 

# Categorical covaraites 
temp_df_cov = df_cov_bc[['FID', 'IID'] + cov_list]
temp_df_cov.index = temp_df_cov['IID']
temp_df_cov = pd.get_dummies(temp_df_cov[cov_list], columns=cov_list, drop_first=True)

# df_reg
df_reg = df_cov_bc[['FID', 'IID'] + qcov_list].copy()
df_reg.dropna(inplace=True)
df_reg.index = df_reg['IID']
df_reg = df_reg.join(temp_df_cov)
PC_list = [x for x in df_reg.columns if 'PC' in x]
df_reg[PC_list] *= np.sqrt(df_reg.shape[0])
df_reg['const'] = 1
df_reg.drop(['FID', 'IID'], axis=1, inplace=True)
df_reg = df_reg.astype(dtype=np.float32)

# Compute residual
for trait in trait_list_bc:
    df_phen = pd.read_csv(OUT_PATH+'/%s.phen'%trait, sep='\t', header=0, index_col=False)
    df_phen.index = df_phen['IID']    
    sample_list = sorted(set(df_phen.index) & set(df_reg.index))
    print('%s, n_sample=%d, n_sample_retain=%d' % (trait, df_phen.shape[0], len(sample_list)))

    temp_df_reg = df_reg.loc[sample_list].copy()
    drop_list = [x for x in temp_df_reg if (temp_df_reg[x].std()<1e-2) & (x!='const')]
    if len(drop_list)>0:
        temp_df_reg = temp_df_reg.drop(columns=drop_list)
        print('    col dropped: %s'%', '.join(drop_list))
        
    v_y = df_phen.loc[sample_list][trait].values.astype(np.float32)
    mat_X = temp_df_reg.values.astype(np.float32)
    n_sample = v_y.shape[0]
    mat_xtx = np.dot(mat_X.T, mat_X)/n_sample
    mat_xty = np.dot(mat_X.T, v_y)/n_sample
    coef_ = np.linalg.solve(mat_xtx, mat_xty).reshape([-1])
    v_y_res = v_y - mat_X.dot(coef_)
    print('    var(v_y)=%0.4g, var(v_y_res)=%0.4g, R_square=%0.4g\n'
          %(v_y.var(), v_y_res.var(), 1- v_y_res.var()/v_y.var()))
    
    temp_df = pd.DataFrame( data = {
        "FID" : sample_list,
        "IID" : sample_list,
        trait : v_y_res / v_y_res.std()
    }
    )
    temp_df.to_csv(OUT_PATH + '/%s.resid.phen'%trait, sep='\t', header=True, index=False)

biochemistry_AlanineAminotransferase, n_sample=271509, n_sample_retain=269977
    var(v_y)=121.2, var(v_y_res)=109.4, R_square=0.09745

biochemistry_Albumin, n_sample=250348, n_sample_retain=248806
    col dropped: biochemistry_dilutionfactor
    var(v_y)=6.7, var(v_y_res)=6.423, R_square=0.04126

biochemistry_AlkalinePhosphatase, n_sample=272704, n_sample_retain=271166
    var(v_y)=508.6, var(v_y_res)=485.6, R_square=0.04538

biochemistry_ApolipoproteinA, n_sample=249045, n_sample_retain=247505
    col dropped: biochemistry_dilutionfactor
    var(v_y)=0.07314, var(v_y_res)=0.06168, R_square=0.1566

biochemistry_ApolipoproteinB, n_sample=272274, n_sample_retain=270740
    var(v_y)=0.05672, var(v_y_res)=0.05605, R_square=0.01188

biochemistry_AspartateAminotransferase, n_sample=270912, n_sample_retain=269389
    var(v_y)=51.85, var(v_y_res)=48.66, R_square=0.06167

biochemistry_Calcium, n_sample=249913, n_sample_retain=248373
    col dropped: biochemistry_dilutionfactor
    var(v_y)=0.0

### Sanity check 

In [12]:
PHEN_FILE = '/n/groups/price/martin/LDSPEC_data/UKBB_trait_ss50K/@.resid.phen'
PHEN_FILE_OLD = '/n/groups/price/martin/LDSPEC_data/UKBB_trait/@.resid.phen'
# PHEN_FILE_OLD = '/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001/UKBB_trait/@.resid.phen'
# PHEN_FILE = "/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/UKBB_trait/@.resid.phen"
# PHEN_FILE_OLD = "/n/groups/price/martin/data_GDREG/WES_50K_imp/UKBB_trait/@.resid.phen"
# for trait in trait_list + trait_list_bc : 
for trait in trait_list_indpt + trait_list_bc_indpt : 
    temp_df1 = pd.read_csv(PHEN_FILE.replace('@', trait), sep='\t', header=0, index_col=False)
    temp_df1.index = temp_df1['IID']
    temp_df2 = pd.read_csv(PHEN_FILE_OLD.replace('@', trait), sep='\t', header=0, index_col=False)
    temp_df2.index = temp_df2['IID']
#     temp_df2.index = [int(x.split('_')[1]) for x in temp_df2['IID']]
    sample_list = sorted(set(temp_df1.index) & set(temp_df2.index))
    print('%s n_sample1=%d, n_sample2=%d, n_overlap=%d'
          % (trait, temp_df1.shape[0], temp_df2.shape[0], len(sample_list)))
    
    temp_df1 = temp_df1.loc[sample_list]
    temp_df2 = temp_df2.loc[sample_list]
    
    print('    cor=%0.3f, mean_abs_err=%0.4g' % (
        np.corrcoef(temp_df1[trait], temp_df2[trait])[0,1], 
        np.absolute(temp_df1[trait]-temp_df2[trait]).mean()
    ))

blood_PLATELET_COUNT n_sample1=48763, n_sample2=326556, n_overlap=48763
    cor=1.000, mean_abs_err=0.02015
blood_RBC_DISTRIB_WIDTH n_sample1=48593, n_sample2=325330, n_overlap=48593
    cor=0.999, mean_abs_err=0.02748
blood_RED_COUNT n_sample1=48835, n_sample2=327144, n_overlap=48835
    cor=1.000, mean_abs_err=0.02264
blood_WHITE_COUNT n_sample1=48780, n_sample2=326659, n_overlap=48780
    cor=1.000, mean_abs_err=0.0207
bmd_HEEL_TSCOREz n_sample1=48965, n_sample2=327675, n_overlap=48965
    cor=1.000, mean_abs_err=0.0175
body_BALDING1 n_sample1=23131, n_sample2=155070, n_overlap=23131
    cor=0.999, mean_abs_err=0.03332
body_BMIz n_sample1=50211, n_sample2=336329, n_overlap=50211
    cor=1.000, mean_abs_err=0.02087
body_HEIGHTz n_sample1=50262, n_sample2=336694, n_overlap=50262
    cor=1.000, mean_abs_err=0.0217
body_WHRadjBMIz n_sample1=50273, n_sample2=336781, n_overlap=50273
    cor=1.000, mean_abs_err=0.02225
bp_DIASTOLICadjMEDz n_sample1=46427, n_sample2=310771, n_overlap=46427
